Below is the coarse layout defined in geopyspark to merge model outputs. Each tile is .0512 degrees by .0512 degrees. need to make a shapely multi line string or polygon bject that can be used to save out tiles as geojsons (only if they intersect an aoi) so that we can use each tile geojson with porder

coarse_layout = gps.LayoutDefinition(gps.Extent(-17.541, -34.845, 51.4766, 37.5518), gps.TileLayout(1348, 1414, 4096, 4096))

In [2]:
from shapely.geometry import Polygon
import pyproj
import numpy as np
import geopandas as gpd

In [4]:
def layout_to_geojson(extent=(-17.541, -34.845, 51.4766, 37.5518), stepsize=.0512, output_name="/dev/data/tile_grid.geojson"):
    """
    :param extent: (lonmin, latmin, lonmax, latmax) checked this by 
    looking at coarse layout in run_geopyspark
    
    :param stepsize: width and height of tile in degrees, checked 
    this by measuring extent of probability image tiles
    
    :returns: a geodataframe where each row is a tile polygon
    """
    
    (lonmin, latmin, lonmax, latmax) = extent
    cols = (lonmax - lonmin)/.0512
    rows = (latmax - latmin)/.0512
    # Top left corner of grid, where we start to build the gdf
    XleftOrigin = lonmin
    XrightOrigin = lonmin + stepsize
    YtopOrigin = latmax
    YbottomOrigin = latmax - stepsize
    polygons = []
    
    for i in range(int(cols)):
        
        Ytop = YtopOrigin
        Ybottom =YbottomOrigin
        for j in range(int(rows)):
            polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
            Ytop = Ytop - stepsize
            Ybottom = Ybottom - stepsize
        XleftOrigin = XleftOrigin + stepsize
        XrightOrigin = XrightOrigin + stepsize

    grid = gpd.GeoDataFrame({'geometry':polygons})
    grid.to_file(output_name, driver = "GeoJSON")
    
layout_to_geojson()

In [17]:
aoi1_polygon = gpd.read_file("../cfg/aois/1.geojson")['geometry'][0]
grid_gdf = gpd.read_file("/dev/data/tile_grid.geojson")

aoi1_grid_geometries = grid_gdf.intersection(aoi1_polygon)
aoi1_grid_geometries= aoi1_grid_geometries[aoi1_grid_geometries.is_empty==False]

test_geom=aoi1_grid_geometries.iloc[6]

gpd.GeoSeries([test_geom]).to_file('test_tile.geojson', driver= "GeoJSON")

In [56]:
gpd.GeoSeries([test_geom]).to_file('test_tile.geojson', driver= "GeoJSON")

ssh for idlist

In [ ]:
porder idlist --input "/home/ubuntu/planet/cloud-free-planet/notebooks/test_tile.geojson" --start "2018-07-01" --end "2018-07-31" --item "PSScene4Band" --asset "analytic_sr" --number 10000 --outfile "/home/ubuntu/planet/cloud-free-planet/notebooks/test_aoi1_id.csv" --cmin 0 --cmax 1 --overlap 0


ssh command for ordering

In [ ]:
porder order --name test-aoi1-month07 --idlist ~/planet/cloud-free-planet/notebooks/test_aoi1_id.txt --item PSScene4Band --asset analytic_sr --boundary test_tile.geojson --aws ~/planet/cloud-free-planet/notebooks/aws.yaml --op aws

In [58]:
from subprocess import Popen, PIPE
p = Popen("porder",stdout=PIPE, stderr=PIPE)
stdout, stderr = p.communicate()

OSError: [Errno 2] No such file or directory

In [6]:
stderr

'usage: porder [-h]\n              {quota,base64,idlist,difflist,idsplit,order,download,multipart,multiproc}\n              ...\nporder: error: too few arguments\n'